<a href="https://colab.research.google.com/github/ashfarhangi/Massive_Storage_and_Big_Data/blob/master/code/3_Spark_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark II
MapReduce, Spark, Real-World example




In [ ]:
import os
!wget https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar -xvf /content/spark-3.0.1-bin-hadoop2.7.tgz
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
# Start A Spark Session
spark = SparkSession \
    .builder \
    .getOrCreate()
words = ["Hello", "Spark", "I am here",'Happe','Spark']
sc.parallelize(words).count()

# Part A


In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.functions import lower

sc = SparkContext.getOrCreate()

In [ ]:
# A helper function to compute the list of words in a line of text
import re
def get_words(line):
    return re.compile('\w+').findall(line)

## WordCount


In [ ]:
!wget -q https://raw.githubusercontent.com/ashfarhangi/Reinforcement-Learning-Trading/master/meta.txt
!wget -q https://raw.githubusercontent.com/ashfarhangi/Reinforcement-Learning-Trading/master/review.txt

In [ ]:
#Test.txt
#function to take input test file and make lower case
def Func(lines):
      lines = lines.lower()
      return lines

#read test file, split into different lines for analyzation
#change test file
lines1 = sc.textFile("test.txt").flatMap(lambda line: line.split(" "))

#apply function to make lowercase
fixed_line = lines1.map(Func)

#Read user input, make lower case
WordIn = input("Word: ").lower()

#count instance of word in each line
CountFind = fixed_line.filter(lambda line: WordIn in get_words(line)).count()

#print result, which is case insensitive
print(CountFind)

Word: TeSt
2


To analyse large datasets using Spark you will load them into Resilient Distributed Datasets (RDDs). There are a number of ways in which you can create RDDs. Use the `parallelize()` function to create one from a Python collection, and use the `textFile()` function to create an RDD from the file `review.txt`. 

## Finding Keywords


In [ ]:
#Question 2

import pyspark.sql.functions as f
from pyspark.sql import SQLContext

#read it meta and review text files
sqlContext = SQLContext(sc)
dfmeta = sqlContext.read.json('meta.txt')
dfdata = sqlContext.read.json('review.txt')

#part 1
###########################################################################################
#add the number of words in a new col
filterDFdata_word = dfdata.withColumn('wordCount', f.size(f.split(f.col('reviewText'), ' ')))

#filter by the number of words greater or equal to 100, order by overall rating
filterDFdata = filterDFdata_word.filter(filterDFdata_word.wordCount > "100").orderBy("overall")

#print and save output to file
filterDFdata.coalesce(1).write.csv("/content/overview_output.csv")

########

#part 2
###########################################################################################
#filter by music category
filterDFmeta = dfmeta.filter(dfmeta.categories == "Music")

#match the filtered music category to asin numbers to get review size
diff_data = filterDFmeta.join(dfdata,filterDFmeta["asin"] == dfdata["asin"],"left")

#add a col with the number of words in the review
wordcnt = diff_data.withColumn('wordCount', f.size(f.split(f.col('reviewText'), ' ')))

#count the number of rows to devide from the total number of words in all reviews
x = diff_data.count()   

#Count total number of words in the columns
Sumcol = wordcnt.agg(f.sum("wordCount")).collect()[0][0]

#take the total number of words and devide by the number of rows
Average_word = Sumcol / x

#Total Average
print(Average_word)


94.2723756312453
